In [2]:
import pandas as pd
import os

# os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf 'spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234' -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import os

from DataLoader import DataLoader


In [3]:
loader = DataLoader(data_set="train")
train_df = loader.train_df

TypeError: DataLoader.__init__() missing 1 required positional argument: 'data_set'

In [ ]:
# Count missing values in each column
missing_value_counts = train_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_df.columns])

# Display the count of missing values for each column
missing_value_counts.show()

In [ ]:
train_df.groupBy("vine").count().show()

In [ ]:
train_df.groupBy("verified_purchase").count().show()

In [ ]:
train_df.groupBy("label").count().show()